In [1]:
from binance.client import Client
from binance.enums import HistoricalKlinesType
from datetime import datetime, timedelta
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_rows', 500)
import requests,json
import plotly.graph_objs as go

In [2]:
# Define column names for the DataFrame
colnames = [
    'time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
    'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
    'Taker buy quote asset volume', 'Ignore'
]

# Get all USDT pairs
BASE_URL = 'https://api.kucoin.com'
resp = requests.get(BASE_URL+'/api/v2/symbols')
ticker_list = json.loads(resp.content)
symbols = [ticker['symbol'] for ticker in ticker_list['data'] if str(ticker['symbol'][-4:]) == 'USDT']


interval = '1d'  # Supports intervals like 1m, 1H, 1D, etc.
start = (datetime.today() - timedelta(days=90)).strftime('%m/%d/%Y 00:00:00')   # Start date
end = datetime.today().strftime('%m/%d/%Y 00:00:00') # End date

client = Client()  # Initialize your Binance API client


In [3]:
dicionario_adx = {
    '0-25': '1. Absent or Weak Trend',
    '25-50': '2. Strong Trend',
    '50-75': '3. Very Strong Trend',
    '75-100': '4. Extremely Strong Trend'
}

def check_adx(value):
    for key, name in dicionario_adx.items():
        range_start, range_end = map(int, key.split('-'))
        if range_start <= value <= range_end:
            return name
    return None

In [4]:
def getting_data(symbol):
    kucoin = requests.get(f'https://api.kucoin.com/api/v1/market/candles?type=1day&symbol={symbol}')
    if kucoin.status_code == 200:  # if result is GOOD
        data = json.loads(kucoin.text)  # load the json response from API
        df = pd.DataFrame(data['data'], columns=['unix', 'price_open', 'price_close', 'price_high', 'price_low', 'volume', 'turnover'])  # create dataframe out of json response
        df['symbol'] = f'{symbol.replace("-", "")}'
        df['time_period_start'] = pd.to_datetime(df['unix'].astype(int), unit='s')
        df.sort_values(by='time_period_start', inplace = True) 
        
  # add a human readable date
    else:
        print('Error on connection')
        pass

    df[['price_open', 'price_high', 'price_low', 'price_close']] = df[['price_open', 'price_high', 'price_low', 'price_close']].map(lambda x: float(x))

    # Calculate ADX, DI+ and DI- manually
    df[['adx', 'dmp', 'dmn']] = ta.adx(df['price_high'], df['price_low'], df['price_close'], length = 14)
    df['adx_trend'] = df['adx'].apply(check_adx)


    # # # Calculate ichimoku cloud  
    df[['leading_span_a', 'leading_span_b', 'conversion_line', 'base_line', 'lagging_span']] = ta.ichimoku(df['price_high'], df['price_low'], df['price_close'])[0]
    df[['macd', 'macd_hist', 'macd_signal']] = ta.macd(df['price_close']).map(lambda x: float(x))
    df['macd_delta'] = df['macd'] - df['macd_signal']

    df[['supertrend_trend', 'supertrend_direction', 'supertrend_long', 'supertrend_short']] = ta.supertrend(df['price_high'], df['price_low'], df['price_close'])

    return df


In [5]:
testing = []

total_items = len(symbols)
for index, crypto in enumerate(symbols[:500], start=1):
    try:
        data = getting_data(crypto)
        if not data.empty:  # Check if the dataframe is not empty before appending
            testing.append(data)
            print(f"Item {index}/{total_items}: {crypto}")
    except:
        pass

if testing:  # Check if the list is not empty before concatenating
    df_concat = pd.concat(testing, ignore_index=True)
else:
    # Handle the case when no data is available
    df_concat = pd.DataFrame()

Item 1/800: AVA-USDT
Item 2/800: MTV-USDT
Item 3/800: KMD-USDT
Item 4/800: RFOX-USDT
Item 5/800: TEL-USDT
Item 6/800: TT-USDT
Item 7/800: AERGO-USDT
Item 8/800: XMR-USDT
Item 9/800: ATOM-USDT
Item 10/800: FTM-USDT
Item 11/800: VSYS-USDT
Item 12/800: CHR-USDT
Item 13/800: COTI-USDT
Item 14/800: BNB-USDT
Item 15/800: ALGO-USDT
Item 16/800: XEM-USDT
Item 17/800: XTZ-USDT
Item 18/800: ZEC-USDT
Item 19/800: ADA-USDT
Item 20/800: BOLT-USDT
Item 21/800: ARPA-USDT
Item 22/800: CHZ-USDT
Item 23/800: NOIA-USDT
Item 24/800: WIN-USDT
Item 25/800: BTT-USDT
Item 26/800: EOSC-USDT
Item 27/800: ONE-USDT
Item 28/800: TOKO-USDT
Item 29/800: LUNA-USDT
Item 30/800: SXP-USDT
Item 31/800: AKRO-USDT
Item 32/800: MAP-USDT
Item 33/800: AMPL-USDT
Item 34/800: DAG-USDT
Item 35/800: KPOL-USDT
Item 36/800: ARX-USDT
Item 37/800: NWC-USDT
Item 38/800: BEPRO-USDT
Item 39/800: VRA-USDT
Item 40/800: KSM-USDT
Item 41/800: DASH-USDT
Item 42/800: SUTER-USDT
Item 43/800: SENSO-USDT
Item 44/800: XDB-USDT
Item 45/800: SYLO-U

In [6]:
today = datetime.today().date()
df_indicators = df_concat.loc[
                (df_concat['adx'] > 25) & 
                (df_concat['time_period_start'].dt.date == today) &

                # Ichimoku com menos preço maior do que linha de conversão e linha de base
                (df_concat['price_low'] > df_concat['conversion_line']) &
                (df_concat['price_low'] > df_concat['base_line']) &

                # MACD histograma maior do que 0 e signal maior do que macd
                (df_concat['macd_hist'] > 0) &
                (df_concat['macd_delta'] > 0.01) &
                (df_concat['supertrend_direction'] == 1)
                ]

## Possibilities of Profit

In [7]:
df_indicators[['symbol', 'time_period_start', 'price_open', 'volume', 'adx', 'adx_trend', 'macd_delta', 'supertrend_direction']].sort_values(by=['adx_trend', 'macd_delta'], ascending=False).reset_index()

,index,symbol,time_period_start,price_open,volume,adx,adx_trend,macd_delta,supertrend_direction
0,13399,AVAXUSDT,2023-12-06,24.550000,519078.61661042,52.645508,3. Very Strong Trend,0.138205,1
1,38399,AVAX3LUSDT,2023-12-06,4.450000,42716.8733,50.854543,3. Very Strong Trend,0.130837,1
2,7699,KCSUSDT,2023-12-06,9.816000,124949.53615005,59.310533,3. Very Strong Trend,0.104756,1
3,30899,DEXEUSDT,2023-12-06,4.281400,13292.9368,57.611536,3. Very Strong Trend,0.056558,1
4,34299,SUPERUSDT,2023-12-06,0.666790,3711177.1087,73.387648,3. Very Strong Trend,0.032114,1
5,39099,TRADEUSDT,2023-12-06,0.824696,334825.803,64.656528,3. Very Strong Trend,0.026783,1
6,30999,MATTERUSDT,2023-12-06,0.213340,1952818.4599,67.970223,3. Very Strong Trend,0.018534,1
7,22499,XAVAUSDT,2023-12-06,0.735200,179637.2129,50.793241,3. Very Strong Trend,0.017317,1
8,19899,MAHAUSDT,2023-12-06,0.407800,288528.5452,51.026871,3. Very Strong Trend,0.010781,1
9,8499,BTCUSDT,2023-12-06,44075.000000,1663.07812575,49.153409,2. Strong Trend,549.694602,1


## Follow up of orders

In [8]:
order_book = pd.read_json('order_book.json').T.reset_index()
order_book['index'] = order_book['index'].replace('-', '', regex=True)
open_orders = order_book['index'].loc[~order_book['sold_date'].notnull()]

In [9]:
df_fup = df_concat.loc[
                (df_concat['symbol'].isin(open_orders) ) & 
                (df_concat['time_period_start'].dt.date == today)
                ]

In [10]:
df_fup[['symbol', 'time_period_start', 'price_open', 'volume', 'adx', 'adx_trend', 'macd_delta', 'supertrend_direction']].sort_values(by=['adx'], ascending=False).reset_index()

,index,symbol,time_period_start,price_open,volume,adx,adx_trend,macd_delta,supertrend_direction
0,48299,VISIONUSDT,2023-12-06,0.007281,18457927.399,74.924907,3. Very Strong Trend,-0.000018,1


In [26]:
def candle_chart(symbol): 
    # Create the candlestick trace for price
    df = df_concat.loc[df_concat['symbol'] == f'{symbol}']
    candlestick = go.Candlestick(
        x=df['time_period_start'],
        open=df['price_open'],
        high=df['price_high'],
        low=df['price_low'],
        close=df['price_close'],
        name='Price'
    )

    # Create the figure for price (candlestick chart)
    fig = go.Figure(data=candlestick)

    # Update layout and add title
    fig.update_layout(
        title='Price Candlestick Chart using Plotly',
        xaxis_title='Date',
        yaxis_title='Price'
    )

    # Show the plot
    fig.show()

candle_chart('AVAXUSDT')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [13]:
def adx_chart(symbol): 
    df = df_concat.loc[df_concat['symbol'] == f'{symbol}']
    
    # Create the ADX trace
    adx_line = go.Scatter(
        x=df['time_period_start'],
        y=df['adx'],
        mode='lines',
        name='ADX',
        line=dict(color='black')  # Color for ADX line
    )

    # Create the DI+ trace
    plus_di_line = go.Scatter(
        x=df['time_period_start'],
        y=df['dmp'],
        mode='lines',
        name='DI+',
        line=dict(color='green')  # Color for DI+
    )

    # Create the DI- trace
    minus_di_line = go.Scatter(
        x=df['time_period_start'],
        y=df['dmn'],
        mode='lines',
        name='DI-',
        line=dict(color='red')  # Color for DI-
    )
    # Create a fixed line for value 20 in light grey
    fixed_line = go.Scatter(
        x=df['time_period_start'],
        y=[20] * len(df),  # Creating a list of 20s to match the length of the DataFrame
        mode='lines',
        name='Fixed Line (20)',
        line=dict(color='lightgrey', dash='dash')  # Color and style for the fixed line
    )

    # Create the figure for ADX chart
    fig = go.Figure(data=[adx_line, plus_di_line, minus_di_line, fixed_line])

    # Update layout and add title
    fig.update_layout(
        title='ADX, DI+ and DI- Line Chart with Fixed Line at 20 using Plotly',
        xaxis_title='Date',
        yaxis_title='Values'
    )

    # Show the plot
    fig.show()

adx_chart('AVAXUSDT')

In [14]:
def ichimoku_chart(symbol): 
    df = df_concat.loc[df_concat['symbol'] == f'{symbol}']
    # Create Ichimoku Cloud chart
    fig2 = go.Figure()

    # Add candlestick chart
    fig2.add_trace(go.Candlestick(x=df['time_period_start'],
                                 open=df['price_open'],
                                 high=df['price_high'],
                                 low=df['price_low'],
                                 close=df['price_close'],
                                 name='Candlestick'))

    # Add Ichimoku Cloud lines
    fig2.add_trace(go.Scatter(x=df['time_period_start'], y=df['conversion_line'], mode='lines', name='Conversion Line', line=dict(color='blue')))
    fig2.add_trace(go.Scatter(x=df['time_period_start'], y=df['base_line'], mode='lines', name='Base Line', line=dict(color='red')))
    fig2.add_trace(go.Scatter(x=df['time_period_start'], y=df['leading_span_a'], mode='lines', name='Leading Span A'))
    fig2.add_trace(go.Scatter(x=df['time_period_start'], y=df['leading_span_b'], mode='lines', name='Leading Span B'))

    # Update layout
    fig2.update_layout(title='Price Candlestick with Ichimoku Cloud Indicators',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      showlegend=True)

    # Display fig2ure
    fig2.show()

ichimoku_chart('AVAXUSDT')